<a href="https://colab.research.google.com/github/DSJR741221/DSJackieR/blob/main/Solucion_Reto_SC_63_JacquelineRichard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Paso 1: Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
# Import SimpleImputer for handling missing values
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Paso 2: Cargar los datos a un dataframe 'data'
data = pd.read_csv('/content/sample_data/bbank_marketing_RETO_DS_AS.csv')

# Paso 3: Información general de la base de datos
print("Información del dataset:")
print(data.info())  # Número de registros, total de variables, tipos de variables, etc.
print("\nDatos perdidos por variable:")
print(data.isnull().sum())  # Muestra la cantidad de datos perdidos

# Paso 4: Transformación de variables categóricas
# Mostramos las variables categóricas
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
numerical_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Usaremos LabelEncoder si hay una variable con pocas categorías, y OneHotEncoder para las variables con muchas categorías
encoder = LabelEncoder()

# Paso 5: Transformación de variables numéricas (si es necesario)
# Comprobamos si hay variables con sesgo (skewness)
#numerical_columns = data.select_dtypes(include=[np.number]).columns

for col in numerical_columns:
    skewness = data[col].skew()
    if abs(skewness) > 1:  # Si el sesgo es mayor que 1, transformamos la variable
        # Replace infinite values with NaN
        data[col] = data[col].replace([np.inf, -np.inf], np.nan)
        # Instead of filling NaN with 0, use the median for imputation before log1p

# Paso 6: Definir la variable de salida y las variables de entrada
X = data.drop('y', axis=1)  # Variables de entrada
y = data['y']  # Variable de salida

# Paso 7: Particionar los datos en entrenamiento, validación y prueba (60%, 20%, 20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define categorical and numerical features
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
numerical_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Create pipelines for numerical and categorical features
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), # Impute missing numerical values with mean
    ('scaler', StandardScaler()), # Scale numerical features
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')), # Impute missing categorical values with most frequent
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore')), # One-hot encode categorical features
])

# Combine pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features),
])

# Apply preprocessing to train, validation, and test data
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

# Paso 8: Aplicar Regresión Logística
# Ajuste de parámetros: por ejemplo, probamos diferentes valores de C (regularización)
logreg = LogisticRegression(max_iter=1000, C=0.5, random_state=42)
logreg.fit(X_train, y_train)

# Predicciones en el conjunto de validación
y_val_pred = logreg.predict(X_val)  # Call the predict method with X_val to get predictions

# Matriz de confusión para el modelo de Regresión Logística
conf_matrix_logreg = confusion_matrix(y_val, y_val_pred)
print("Matriz de Confusión - Regresión Logística:")
print(conf_matrix_logreg)

# Clasificación
print("Reporte de clasificación - Regresión Logística:")
print(classification_report(y_val, y_val_pred))

# Ajuste de parámetros: por ejemplo, probamos diferentes valores de C (regularización)
logreg = LogisticRegression(max_iter=1000, C=0.5, random_state=42)
logreg.fit(X_train, y_train)

# Predicciones en el conjunto de validación
y_val_pred = logreg.predict(X_val)
conf_matrix_logreg = confusion_matrix(y_val, y_val_pred)
print("Matriz de Confusión Ajustada - Regresión Logística:")
print(conf_matrix_logreg)

# Paso 9: Aplicar Red Neuronal
mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

# Predicciones en el conjunto de validación
y_val_pred_mlp = mlp.predict(X_val)

# Matriz de confusión para el modelo de Red Neuronal
conf_matrix_mlp = confusion_matrix(y_val, y_val_pred_mlp)
print("Matriz de Confusión - Red Neuronal:")
print(conf_matrix_mlp)

# Ajuste de parámetros: probamos diferentes combinaciones de neuronas y capas ocultas
mlp = MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
y_val_pred_mlp = mlp.predict(X_val)
conf_matrix_mlp = confusion_matrix(y_val, y_val_pred_mlp)
print("Matriz de Confusión Ajustada - Red Neuronal:")
print(conf_matrix_mlp)

# Paso 10: Desempeño final del modelo en el conjunto de prueba
best_model = logreg  # Supongamos que el mejor modelo es la regresión logística

# Predicciones en el conjunto de prueba
y_test_pred = best_model.predict(X_test)

# Matriz de confusión final
final_conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Matriz de Confusión Final:")
print(final_conf_matrix)

# Reporte de clasificación final
print("Reporte de clasificación Final:")
print(classification_report(y_test, y_test_pred))


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        9000 non-null   int64 
 1   job        9000 non-null   object
 2   marital    9000 non-null   object
 3   education  9000 non-null   object
 4   default    9000 non-null   object
 5   balance    9000 non-null   int64 
 6   housing    9000 non-null   object
 7   loan       9000 non-null   object
 8   contact    9000 non-null   object
 9   day        9000 non-null   int64 
 10  month      9000 non-null   object
 11  duration   9000 non-null   int64 
 12  campaign   9000 non-null   int64 
 13  pdays      9000 non-null   int64 
 14  previous   9000 non-null   int64 
 15  poutcome   9000 non-null   object
 16  y          9000 non-null   object
dtypes: int64(7), object(10)
memory usage: 1.2+ MB
None

Datos perdidos por variable:
age          0
job          0
marital    